In [1]:
import pandas as pd
import quandl
import math
import numpy as np
from sklearn import preprocessing,model_selection, svm
from sklearn.linear_model import LinearRegression

In [3]:
quandl.ApiConfig.api_key="bsZzMiC7v9JnsD46JhM3"
df = quandl.get('WIKI/GOOGL')

In [6]:
df.tail()

,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
Date,,,,,,,,,,,,
2017-10-02,975.65,977.7400,961.950,967.47,1466444.0,0.0,1.0,975.65,977.7400,961.950,967.47,1466444.0
2017-10-03,967.56,972.4400,962.705,972.08,1080312.0,0.0,1.0,967.56,972.4400,962.705,972.08,1080312.0
2017-10-04,971.76,974.4000,965.610,966.78,896531.0,0.0,1.0,971.76,974.4000,965.610,966.78,896531.0
2017-10-05,972.79,986.5085,970.270,985.19,1627255.0,0.0,1.0,972.79,986.5085,970.270,985.19,1627255.0
2017-10-06,980.00,994.2600,978.510,993.64,1490744.0,0.0,1.0,980.00,994.2600,978.510,993.64,1490744.0


In [7]:
df['HL_PCT'] = (df['Adj. High'] - df['Adj. Close']) / df['Adj. Close'] * 100
df['PCT_change'] = (df['Adj. Close'] - df['Adj. Open']) / df['Adj. Open'] * 100

In [9]:
df = df[['Adj. Close', 'HL_PCT', 'PCT_change','Adj. Volume']]
df.tail()

,Adj. Close,HL_PCT,PCT_change,Adj. Volume
Date,,,,
2017-10-02,967.47,1.061532,-0.838415,1466444.0
2017-10-03,972.08,0.037034,0.467154,1080312.0
2017-10-04,966.78,0.788183,-0.512472,896531.0
2017-10-05,985.19,0.133832,1.274684,1627255.0
2017-10-06,993.64,0.062397,1.391837,1490744.0


In [10]:
forecast_col='Adj. Close' 

In [11]:
df.fillna(-99999, inplace=True)

In [12]:
forecast_out= int(math.ceil(0.01*len(df))) # 1% of the DataFrame

In [13]:
df['label']=df[forecast_col].shift(-forecast_out)
# this way the label column for each row will be...
# ... the adjusted price of 100 days into the future

In [14]:
df.head(50)

,Adj. Close,HL_PCT,PCT_change,Adj. Volume,label
Date,,,,,
2004-08-19,50.322842,3.712563,0.324968,44659000.0,69.639972
2004-08-20,54.322689,0.710922,7.227007,22834300.0,69.078238
2004-08-23,54.869377,3.729433,-1.227880,18256100.0,67.839414
2004-08-24,52.597363,6.417469,-5.726357,15247300.0,68.912727
2004-08-25,53.164113,1.886792,1.183658,9188600.0,70.668146
2004-08-26,54.122070,0.037068,2.820391,7094800.0,71.219849
2004-08-27,53.239345,2.326896,-1.803885,6211700.0,72.278116
2004-08-30,51.162935,3.411430,-3.106003,5196700.0,74.810934
2004-08-31,51.343492,1.308977,0.048866,4917800.0,74.199045
